In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import quandl
import cvxpy as cvx

In [3]:
sys.path.insert(0, os.path.abspath('..'))
import helium

In [4]:
%matplotlib inline

In [5]:
quandl.ApiConfig.api_key = '8wnB7EcvJm1xEvJjzWBR'

In [6]:
tickers = ['AMZN', 'GOOGL', 'MCD', 'NKE']
start_date='2016-01-01'
end_date='2017-01-13'
prices = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'])
                for ticker in tickers]))

returns=prices.pct_change()
returns[["_CASH"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,MCD,NKE,_CASH
Date,,,,,
2017-01-09,0.001168,0.002387,-0.002733,-0.009831,0.000020
2017-01-10,-0.001280,-0.001414,-0.001495,-0.005058,0.000020
2017-01-11,0.003920,0.004661,0.005239,-0.007908,0.000020
2017-01-12,0.018297,-0.000398,0.010093,-0.005504,0.000020
2017-01-13,0.004302,0.001700,-0.004914,0.009924,0.000021


In [7]:
rets = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
sigmas = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack()

rets.tail()

,AMZN,GOOGL,MCD,NKE,_CASH
Date,,,,,
2017-01-09,0.001182,0.000553,0.000304,-0.000252,0.000013
2017-01-10,0.001185,0.000496,0.000267,-0.000317,0.000013
2017-01-11,0.001414,0.000628,0.000340,-0.000260,0.000013
2017-01-12,0.001352,0.000581,0.000309,-0.000273,0.000013
2017-01-13,0.001580,0.000694,0.000399,-0.000230,0.000013


In [8]:
cash_ticker = "_CASH"

In [9]:
price_sigma = rets.copy()
price_sigma[:] = 0.

In [10]:
volumes = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Volume'])
                for ticker in tickers]))
volumes[cash_ticker] = np.finfo(np.float).max

In [11]:
tcost_model=helium.TransactionCost(gamma = 1., 
                                   half_spread=10E-4,
                                   nonlin_coef = 0.,  #1.,
                                   sigmas = price_sigma, 
                                   nonlin_power = 1.5,
                                   volumes = volumes,
                                   asym_coef = 0.)

In [12]:
borrow_costs = rets.copy()
borrow_costs[:] = 1E-4
borrow_costs[cash_ticker] = 0.

dividends = rets.copy()
dividends[:] = 0.


hcost_model=helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)

In [13]:
risk_model = helium.BasicRiskCost(5.0, sigmas)

In [14]:
leverage_limit = helium.LeverageLimitConstraint(3.)

In [15]:
holding_init = pd.Series(index=rets.columns, data=0.)
holding_init[cash_ticker] = 10000.
holding_init

AMZN         0.0
GOOGL        0.0
MCD          0.0
NKE          0.0
_CASH    10000.0
dtype: float64

In [16]:
deltas = rets.copy()
deltas[:]  = 0.

In [17]:
r = helium.DefaultRet(rets=rets, deltas = deltas, gamma_decay=None)

In [18]:
spo_policy = helium.SinglePeriodOpt(rets=r,
                                   costs=[risk_model, hcost_model, tcost_model],
                                   constraints=[leverage_limit]
                                   )

In [19]:
shares_to_trade=spo_policy.get_trades(h=holding_init, t=end_date)
shares_to_trade

AMZN     1793.597144
GOOGL       0.000012
MCD         0.000008
NKE        -0.000005
_CASH   -1793.597160
dtype: float64

In [20]:
shares_to_trade

AMZN     1793.597144
GOOGL       0.000012
MCD         0.000008
NKE        -0.000005
_CASH   -1793.597160
dtype: float64

In [21]:
np.round(shares_to_trade/prices.loc[end_date])

AMZN     2.0
GOOGL    0.0
MCD      0.0
NKE     -0.0
_CASH    NaN
dtype: float64